<a href="https://colab.research.google.com/github/tanishajha2101-eng/Attention-Based-Hybrid-CNN-Thermography/blob/main/Copy_of_Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print("TF version:", tf.__version__)
print("GPU:", tf.config.list_physical_devices('GPU'))


TF version: 2.19.0
GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
import os

base_path = "/content/drive/MyDrive"
print("Base exists:", os.path.exists(base_path))
print("Folders:", os.listdir(base_path)[:20])


Base exists: True
Folders: ['Colab Notebooks', 'BCD_Thermography', 'BCD_Thermography_split', 'Breast_Cancer_Dataset']


In [ ]:
import os

for root, dirs, files in os.walk("/content/drive/MyDrive"):
    if "NORMAL" in dirs and "SICK" in dirs:
        print("✅ Dataset Found at:", root)
        break


✅ Dataset Found at: /content/drive/MyDrive/BCD_Thermography


In [ ]:
DATASET_PATH = "/content/drive/MyDrive/BCD_Thermography"


In [ ]:
import os
print("NORMAL exists:", os.path.exists(DATASET_PATH + "/NORMAL"))
print("SICK exists:", os.path.exists(DATASET_PATH + "/SICK"))
print("NORMAL images:", len(os.listdir(DATASET_PATH + "/NORMAL")))
print("SICK images:", len(os.listdir(DATASET_PATH + "/SICK")))


NORMAL exists: True
SICK exists: True
NORMAL images: 162
SICK images: 100


In [ ]:
import os
print(os.listdir("/content/drive/MyDrive")[:50])


['Colab Notebooks', 'BCD_Thermography', 'BCD_Thermography_split', 'Breast_Cancer_Dataset']


In [ ]:
import os
print("Dataset folder exists:", os.path.exists("/content/drive/MyDrive/BCD_Thermography"))


Dataset folder exists: True


In [ ]:
import os
print(os.listdir("/content/drive/MyDrive"))


['Colab Notebooks', 'BCD_Thermography', 'BCD_Thermography_split', 'Breast_Cancer_Dataset']


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
import os

DATASET_PATH = "/content/drive/MyDrive/BCD_Thermography"

print("Dataset folder exists:", os.path.exists(DATASET_PATH))
print("NORMAL exists:", os.path.exists(DATASET_PATH + "/NORMAL"))
print("SICK exists:", os.path.exists(DATASET_PATH + "/SICK"))

print("NORMAL images:", len(os.listdir(DATASET_PATH + "/NORMAL")))
print("SICK images:", len(os.listdir(DATASET_PATH + "/SICK")))


Dataset folder exists: True
NORMAL exists: True
SICK exists: True
NORMAL images: 162
SICK images: 100


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224,224)
BATCH_SIZE = 16

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training",
    shuffle=True,
    classes=["NORMAL","SICK"]
)

val_gen = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation",
    shuffle=False,
    classes=["NORMAL","SICK"]
)


Found 210 images belonging to 2 classes.
Found 52 images belonging to 2 classes.


In [ ]:
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3),
    pooling="avg"
)
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15
)


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 128s 7s/step - accuracy: 0.6124 - auc: 0.3971 - loss: 0.6865 - val_accuracy: 0.6154 - val_auc: 0.5000 - val_loss: 0.6677
Epoch 2/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 273ms/step - accuracy: 0.6034 - auc: 0.3948 - loss: 0.6830 - val_accuracy: 0.6154 - val_auc: 0.4750 - val_loss: 0.6668
Epoch 3/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 276ms/step - accuracy: 0.6290 - auc: 0.5018 - loss: 0.6648 - val_accuracy: 0.6154 - val_auc: 0.4375 - val_loss: 0.6664
Epoch 4/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 397ms/step - accuracy: 0.5785 - auc: 0.3545 - loss: 0.7095 - val_accuracy: 0.6154 - val_auc: 0.4563 - val_loss: 0.6666
Epoch 5/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 274ms/step - accuracy: 0.5831 - auc: 0.4946 - loss: 0.6829 - val_accuracy: 0.6154 - val_auc: 0.5000 - val_loss: 0.6665
Epoch 6/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 275ms/step - accuracy: 0.5999 - auc: 0.4946 - loss: 0.6785 - val_accuracy: 0.6154 - val_auc: 0.3906 - val_loss: 0.6664
Epoch 7/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

val_gen.reset()
y_true = val_gen.classes
y_prob = model.predict(val_gen).ravel()
y_pred = (y_prob > 0.5).astype(int)

print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("\nReport:\n", classification_report(y_true, y_pred, target_names=["NORMAL","SICK"]))


4/4 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step
Confusion Matrix:
 [[32  0]
 [20  0]]

Report:
               precision    recall  f1-score   support

      NORMAL       0.62      1.00      0.76        32
        SICK       0.00      0.00      0.00        20

    accuracy                           0.62        52
   macro avg       0.31      0.50      0.38        52
weighted avg       0.38      0.62      0.47        52



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import os

DATASET_PATH = "/content/drive/MyDrive/BCD_Thermography"
print("Exists:", os.path.exists(DATASET_PATH))
print("Folders:", os.listdir(DATASET_PATH))
print("NORMAL count:", len(os.listdir(DATASET_PATH+"/NORMAL")))
print("SICK count:", len(os.listdir(DATASET_PATH+"/SICK")))


Exists: True
Folders: ['NORMAL', 'SICK']
NORMAL count: 162
SICK count: 100


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_true = val_gen.classes
y_prob = model.predict(val_gen).ravel()

for t in [0.1,0.2,0.3,0.4,0.5]:
    y_pred = (y_prob > t).astype(int)
    print("\nThreshold =", t)
    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred, target_names=["NORMAL","SICK"]))


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 168ms/step

Threshold = 0.1
[[ 0 32]
 [ 0 20]]
              precision    recall  f1-score   support

      NORMAL       0.00      0.00      0.00        32
        SICK       0.38      1.00      0.56        20

    accuracy                           0.38        52
   macro avg       0.19      0.50      0.28        52
weighted avg       0.15      0.38      0.21        52


Threshold = 0.2
[[ 0 32]
 [ 0 20]]
              precision    recall  f1-score   support

      NORMAL       0.00      0.00      0.00        32
        SICK       0.38      1.00      0.56        20

    accuracy                           0.38        52
   macro avg       0.19      0.50      0.28        52
weighted avg       0.15      0.38      0.21        52


Threshold = 0.3
[[ 0 32]
 [ 0 20]]
              precision    recall  f1-score   support

      NORMAL       0.00      0.00      0.00        32
        SICK       0.38      1.00      0.56        20

    accuracy                       

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/m

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224,224,3))
base.trainable = False   # IMPORTANT

x = GlobalAveragePooling2D()(base.output)
x = Dropout(0.3)(x)
out = Dense(1, activation="sigmoid")(x)

model = Model(base.input, out)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    class_weight=class_weight
)


NameError: name 'class_weight' is not defined

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)

class_weight = {0: weights[0], 1: weights[1]}
print("Class weight:", class_weight)


In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    class_weight=class_weight
)


In [ ]:
base_model.trainable = False


In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetB0

IMG_SIZE = (224,224)

base_model = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3)
)

base_model.trainable = False

inputs = tf.keras.Input(shape=(224,224,3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

model.summary()


In [ ]:
history1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    class_weight=class_weight
)


In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    class_weight=class_weight
)


In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

val_gen.reset()
y_true = val_gen.classes
y_prob = model.predict(val_gen).ravel()

# try threshold 0.5, 0.4, 0.3
thr = 0.4
y_pred = (y_prob > thr).astype(int)

print("Threshold:", thr)
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("\nReport:\n", classification_report(y_true, y_pred, target_names=["NORMAL","SICK"]))


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


In [ ]:
import os
print(os.listdir("/content/drive/MyDrive"))


In [ ]:
import os
print("Drive exists:", os.path.exists("/content/drive"))
print("MyDrive exists:", os.path.exists("/content/drive/MyDrive"))


In [ ]:
import os
print(os.listdir("/content/drive/MyDrive"))


In [ ]:
DATASET_PATH = "/content/drive/MyDrive/BCD_Thermography"
import os
print("Exists:", os.path.exists(DATASET_PATH))
print("Folders inside:", os.listdir(DATASET_PATH))


In [ ]:
print("NORMAL count:", len(os.listdir(DATASET_PATH + "/NORMAL")))
print("SICK count:", len(os.listdir(DATASET_PATH + "/SICK")))


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224,224)
BATCH_SIZE = 16

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training",
    shuffle=True,
    classes=["NORMAL","SICK"]
)

val_gen = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation",
    shuffle=False,
    classes=["NORMAL","SICK"]
)

print("Train samples:", train_gen.samples)
print("Val samples:", val_gen.samples)
print("Class indices:", train_gen.class_indices)


In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

base_model = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3),
    pooling="avg"
)

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.Dropout(0.3),
    layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

callbacks = [
    EarlyStopping(monitor="val_auc", patience=4, mode="max", restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", patience=2, factor=0.5, verbose=1)
]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    callbacks=callbacks
)


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

val_gen.reset()
y_true = val_gen.classes
y_prob = model.predict(val_gen).ravel()
y_pred = (y_prob > 0.5).astype(int)

print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=["NORMAL","SICK"]))


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)

class_weight = {0: weights[0], 1: weights[1]}
print("class_weight:", class_weight)


In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    callbacks=callbacks,
    class_weight=class_weight
)


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

val_gen.reset()
y_true = val_gen.classes
y_prob = model.predict(val_gen).ravel()

for t in [0.2,0.3]:
    y_pred = (y_prob > t).astype(int)
    print("\nThreshold =", t)
    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred, target_names=["NORMAL","SICK"]))


In [ ]:
base_model.trainable = True

for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=callbacks,
    class_weight=class_weight
)


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


In [ ]:
import os

BASE = "/content/drive/MyDrive/BCD_Thermography_split"
TRAIN_PATH = BASE + "/train"
VAL_PATH = BASE + "/val"

print("BASE exists:", os.path.exists(BASE))
print("TRAIN exists:", os.path.exists(TRAIN_PATH))
print("VAL exists:", os.path.exists(VAL_PATH))

print("Train folders:", os.listdir(TRAIN_PATH))
print("Val folders:", os.listdir(VAL_PATH))

print("Train NORMAL:", len(os.listdir(TRAIN_PATH + "/NORMAL")))
print("Train SICK:", len(os.listdir(TRAIN_PATH + "/SICK")))
print("Val NORMAL:", len(os.listdir(VAL_PATH + "/NORMAL")))
print("Val SICK:", len(os.listdir(VAL_PATH + "/SICK")))


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224,224)
BATCH_SIZE = 16

TRAIN_PATH = "/content/drive/MyDrive/BCD_Thermography_split/train"
VAL_PATH   = "/content/drive/MyDrive/BCD_Thermography_split/val"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=True,
    classes=["NORMAL","SICK"]
)

val_gen = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False,
    classes=["NORMAL","SICK"]
)

print("Train samples:", train_gen.samples)
print("Val samples:", val_gen.samples)
print("Class indices:", train_gen.class_indices)


In [ ]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

classes = np.unique(train_gen.classes)
weights = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=train_gen.classes
)
class_weight = {int(c): w for c, w in zip(classes, weights)}
print("class_weight:", class_weight)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3)
)

base_model.trainable = False  # pehle freeze

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

model.summary()


In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_auc",
        mode="max",
        patience=5,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=2,
        min_lr=1e-7
    ),
    tf.keras.callbacks.ModelCheckpoint(
        "best_model.h5",
        monitor="val_auc",
        mode="max",
        save_best_only=True
    )
]


In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=callbacks,
    class_weight=class_weight
)


In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')


In [ ]:
loss_fn = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=loss_fn,
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)


In [ ]:
base_model.trainable = False
history1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=8,
    class_weight=class_weight,
    callbacks=callbacks
)


In [ ]:
base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05),
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    class_weight=class_weight,
    callbacks=callbacks
)


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.0,
    brightness_range=(0.8, 1.2),
    zoom_range=0.1
)


In [ ]:
val_gen.reset()
y_prob = model.predict(val_gen).ravel()
print("Min prob:", y_prob.min())
print("Max prob:", y_prob.max())
print("Mean prob:", y_prob.mean())


In [ ]:
from tensorflow.keras.layers import Dense


In [ ]:
globals().keys()


In [ ]:
import os
print(os.listdir("/content"))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
base_path = "/content/drive/MyDrive/BCD_Thermography_split"


In [ ]:
import os
print(os.listdir(base_path))


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224,224)
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    base_path + "/train",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    base_path + "/val",   # ⚠️ agar val folder ka naam "valid" ho toh change karna
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    shuffle=False
)


In [ ]:
print("Train classes:", train_gen.class_indices)
print("Val classes:", val_gen.class_indices)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

base_model = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3)
)

base_model.trainable = False  # pehle freeze

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

early = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", patience=2, factor=0.5, verbose=1)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=[early, reduce_lr]
)


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

y_train = train_gen.classes

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)

class_weight_dict = {i:w for i,w in enumerate(class_weights)}
print("Class weights:", class_weight_dict)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    class_weight=class_weight_dict,
    callbacks=[early, reduce_lr]
)


In [ ]:
base_model.trainable = True

# last 30 layers train karo, baaki freeze
for layer in base_model.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    class_weight=class_weight_dict,
    callbacks=[early, reduce_lr]
)


In [ ]:
batch_size = 8


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x_batch, y_batch = next(train_gen)

plt.figure(figsize=(10,5))
for i in range(8):
    plt.subplot(2,4,i+1)
    plt.imshow(x_batch[i])
    plt.title("SICK" if y_batch[i]==1 else "NORMAL")
    plt.axis("off")
plt.show()


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,      # 20 → 5
    zoom_range=0.1,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=False  # OFF
)


In [ ]:
train_gen.reset()
val_gen.reset()


In [ ]:
base_model.trainable = True

for layer in base_model.layers[:-40]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    class_weight=class_weight_dict,
    callbacks=[early, reduce_lr]
)


In [ ]:

import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

val_gen.reset()
y_prob = model.predict(val_gen).ravel()
y_pred = (y_prob > 0.5).astype(int)
y_true = val_gen.classes

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=["NORMAL","SICK"]))


In [ ]:
import numpy as np
from sklearn.metrics import f1_score

val_gen.reset()
y_prob = model.predict(val_gen).ravel()
y_true = val_gen.classes

best_t, best_f1 = 0, 0
for t in np.arange(0.1, 0.9, 0.05):
    y_pred = (y_prob > t).astype(int)
    f1 = f1_score(y_true, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_t = t

print("Best threshold:", best_t)
print("Best F1:", best_f1)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = (y_prob > best_t).astype(int)
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=["NORMAL","SICK"]))


In [ ]:
base_model.trainable = True

for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history3 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    class_weight=class_weight_dict,
    callbacks=[early, reduce_lr]
)


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)


In [ ]:
class_weight_dict = {0: 0.8, 1: 2.0}


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

base = MobileNetV2(include_top=False, weights="imagenet", input_shape=(224,224,3))
base.trainable = False

x = GlobalAveragePooling2D()(base.output)
x = Dropout(0.3)(x)
out = Dense(1, activation="sigmoid")(x)

model = Model(base.input, out)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

early = EarlyStopping(monitor="val_auc", mode="max", patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", patience=2, factor=0.5, verbose=1)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    class_weight={0:0.8, 1:2.0},
    callbacks=[early, reduce_lr]
)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

base = MobileNetV2(include_top=False, weights="imagenet", input_shape=(224,224,3))
base.trainable = False

x = GlobalAveragePooling2D()(base.output)
x = Dropout(0.3)(x)
out = Dense(1, activation="sigmoid")(x)

model = Model(base.input, out)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

early = EarlyStopping(monitor="val_auc", mode="max", patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", patience=2, factor=0.5, verbose=1)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    class_weight={0:0.8, 1:2.0},
    callbacks=[early, reduce_lr]
)


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

val_gen.reset()
y_prob = model.predict(val_gen).ravel()
y_pred = (y_prob > 0.5).astype(int)
y_true = val_gen.classes

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=["NORMAL","SICK"]))
print("Prob min/max/mean:", y_prob.min(), y_prob.max(), y_prob.mean())


In [ ]:
base.trainable = True
for layer in base.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    class_weight={0:0.8, 1:2.0},
    callbacks=[early, reduce_lr]
)


In [ ]:
import numpy as np
from sklearn.metrics import f1_score

val_gen.reset()
y_prob = model.predict(val_gen).ravel()
y_true = val_gen.classes

for t in [0.4,0.5,0.55,0.6,0.65,0.7]:
    y_pred = (y_prob > t).astype(int)
    f1 = f1_score(y_true, y_pred)
    print("Threshold:", t, "F1:", f1)


In [ ]:
class_weight={0:1.0, 1:1.5}


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

for t in [0.5, 0.6]:
    y_pred = (y_prob > t).astype(int)
    print("\nThreshold:", t)
    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred, target_names=["NORMAL","SICK"]))


In [ ]:
history_new = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    class_weight={0:1.0, 1:1.5},
    callbacks=[early, reduce_lr]
)


In [ ]:
base.trainable = True
for layer in base.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

history_ft = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    class_weight={0:1.0, 1:1.5},
    callbacks=[early, reduce_lr]
)


In [ ]:
early


In [ ]:
EarlyStopping(monitor="val_auc", mode="max", patience=5, restore_best_weights=True)


In [ ]:
early = EarlyStopping(
    monitor="val_auc",
    mode="max",
    patience=10,   # 5 -> 10
    restore_best_weights=True
)


In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    class_weight={0:1.0, 1:1.5},
    callbacks=[reduce_lr]   # early removed
)


In [ ]:
print("Best val_auc:", max(history_ft.history["val_auc"]))
print("Best val_acc:", max(history_ft.history["val_accuracy"]))


In [ ]:
val_gen.reset()
loss, acc, auc = model.evaluate(val_gen, verbose=0)
print("VAL Loss:", loss)
print("VAL Accuracy:", acc)
print("VAL AUC:", auc)


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

val_gen.reset()
y_prob = model.predict(val_gen).ravel()
y_true = val_gen.classes

best_t, best_acc = 0, 0
for t in np.arange(0.3, 0.9, 0.05):
    y_pred = (y_prob > t).astype(int)
    acc = accuracy_score(y_true, y_pred)
    if acc > best_acc:
        best_acc = acc
        best_t = t

print("Best threshold for ACC:", best_t)
print("Best ACC:", best_acc)


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = (y_prob > best_t).astype(int)
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=["NORMAL","SICK"]))


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.05
)


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.05
)


In [ ]:
class_weight = {0:1.0, 1:1.2}


In [ ]:
train_gen = train_datagen.flow_from_directory(
    base_path + "/train",
    target_size=(224,224),
    batch_size=8,
    class_mode="binary",
    shuffle=True
)

val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    base_path + "/val",
    target_size=(224,224),
    batch_size=8,
    class_mode="binary",
    shuffle=False
)


In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    class_weight={0:1.0, 1:1.2},
    callbacks=[early, reduce_lr]
)


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

val_gen.reset()
y_prob = model.predict(val_gen).ravel()
y_true = val_gen.classes

best_t, best_acc = 0, 0
for t in np.arange(0.3, 0.95, 0.05):
    y_pred = (y_prob > t).astype(int)
    acc = accuracy_score(y_true, y_pred)
    if acc > best_acc:
        best_acc = acc
        best_t = t

print("Best threshold:", best_t)
print("Best accuracy:", best_acc)

y_pred = (y_prob > best_t).astype(int)
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=["NORMAL","SICK"]))


In [ ]:
import numpy as np
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

val_gen.reset()
y_prob = model.predict(val_gen).ravel()
y_true = val_gen.classes

best_t, best_bacc = 0, 0
for t in np.arange(0.1, 0.95, 0.05):
    y_pred = (y_prob > t).astype(int)
    bacc = balanced_accuracy_score(y_true, y_pred)
    if bacc > best_bacc:
        best_bacc = bacc
        best_t = t

print("Best threshold for Balanced Acc:", best_t)
print("Best Balanced Accuracy:", best_bacc)

y_pred = (y_prob > best_t).astype(int)
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=["NORMAL","SICK"]))


In [ ]:
from sklearn.metrics import accuracy_score

best_t, best_acc = 0, 0
for t in np.arange(0.3, 0.75, 0.05):
    y_pred = (y_prob > t).astype(int)
    acc = accuracy_score(y_true, y_pred)
    if acc > best_acc:
        best_acc = acc
        best_t = t

print("Best threshold (restricted):", best_t)
print("Best ACC (restricted):", best_acc)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

IMG_SIZE = (224,224)

base = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3)
)
base.trainable = False   # first freeze

inputs = layers.Input(shape=(224,224,3))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)


In [ ]:
class_weight = {0: 1.0, 1: 1.62}


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.05
)
val_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
history1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    class_weight=class_weight
)


In [ ]:
val_gen.reset()
loss, acc, auc = model.evaluate(val_gen, verbose=0)
print("VAL acc:", acc)
print("VAL auc:", auc)


In [ ]:
import numpy as np
from sklearn.metrics import roc_curve, confusion_matrix, classification_report

val_gen.reset()
y_prob = model.predict(val_gen).ravel()
y_true = val_gen.classes

fpr, tpr, thresholds = roc_curve(y_true, y_prob)
best_idx = np.argmax(tpr - fpr)   # Youden’s J statistic
best_t = thresholds[best_idx]

print("Best threshold (Youden):", best_t)

y_pred = (y_prob > best_t).astype(int)
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=["NORMAL","SICK"]))


In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    zoom_range=0.05
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)


In [ ]:
base_path = "/content/BCD_Therography_split"


In [ ]:
base_path = "/content/BCD_Thermography_split"


In [ ]:
import os
print(os.listdir("/content"))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os
print(os.listdir("/content/drive/MyDrive"))


In [ ]:
base_path = "/content/drive/MyDrive/BCD_Thermography_split"


In [ ]:
import os
print(os.listdir(base_path))


In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224,224)
batch_size = 8

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    zoom_range=0.05
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_gen = train_datagen.flow_from_directory(
    base_path + "/train",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    base_path + "/val",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    shuffle=False
)

print("Train:", train_gen.class_indices)
print("Val:", val_gen.class_indices)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

base = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3)
)
base.trainable = False

inputs = layers.Input(shape=(224,224,3))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

class_weight = {0:1.0, 1:1.62}

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    class_weight=class_weight
)


In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


In [ ]:
class_weight = {0:1.0, 1:1.3}


In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)


In [ ]:
model.summary()


In [ ]:
print(base_model.name)


In [ ]:
tf.keras.applications.MobileNetV2


In [ ]:
tf.keras.applications.EfficientNetB0


In [ ]:
tf.keras.applications.ResNet50


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np


In [ ]:
img_size = (224,224)
batch_size = 8
base_path = "/content/drive/MyDrive/BCD_Thermography_split"

train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

train_gen = train_datagen.flow_from_directory(
    base_path + "/train",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    base_path + "/val",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    shuffle=False
)

print("Train:", train_gen.class_indices)
print("Val:", val_gen.class_indices)


In [ ]:
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3)
)

base_model.trainable = False  # Phase 1 freeze

inputs = layers.Input(shape=(224,224,3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

model.summary()
print("Base model:", base_model.name)


In [ ]:
early = EarlyStopping(
    monitor="val_auc",
    mode="max",
    patience=4,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_auc",
    mode="max",
    patience=2,
    factor=0.5,
    min_lr=1e-7,
    verbose=1
)


In [ ]:
history1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=8,
    callbacks=[early, reduce_lr]
)


In [ ]:
base_model.trainable = True

# Unfreeze last 40 layers only
for layer in base_model.layers[:-40]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    callbacks=[early, reduce_lr]
)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

val_gen.reset()
y_prob = model.predict(val_gen).ravel()
y_true = val_gen.classes

# default threshold 0.5
y_pred = (y_prob > 0.5).astype(int)

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=["NORMAL","SICK"]))



In [ ]:
early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=8,
    restore_best_weights=True
)


In [ ]:
early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=8,
    restore_best_weights=True
)


In [ ]:
print(base_model.name)
model.summary()


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input

img_size = (224,224)
batch_size = 8
base_path = "/content/drive/MyDrive/BCD_Thermography_split"

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_gen = train_datagen.flow_from_directory(
    base_path + "/train",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    base_path + "/val",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    shuffle=False
)


In [ ]:
base_model = EfficientNetB0(include_top=False, weights="imagenet", input_shape=(224,224,3))
base_model.trainable = False

inputs = tf.keras.Input(shape=(224,224,3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True)

history1 = model.fit(train_gen, validation_data=val_gen, epochs=15, callbacks=[early])


In [ ]:
base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

history2 = model.fit(train_gen, validation_data=val_gen, epochs=15, callbacks=[early])


In [ ]:
class_weight = {0:1.0, 1:1.3}


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

val_gen.reset()
y_prob = model.predict(val_gen).ravel()
y_true = val_gen.classes

y_pred = (y_prob > 0.5).astype(int)

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=["NORMAL","SICK"]))


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

# EfficientNetB0 base
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3)
)

base_model.trainable = False   # pehle freeze

inputs = layers.Input(shape=(224,224,3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

model.summary()


In [ ]:

print(base_model.name)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input


In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    shear_range=0.1,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True
)


In [ ]:
val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)


In [ ]:
base_path = "/content/drive/MyDrive/Colab Notebooks/BCD_Thermography_split"


In [ ]:
base_path = "/content/drive/MyDrive/Colab Notebooks/BCD_Thermography_split"


In [ ]:
base_path = "/content/drive/MyDrive/BCD_Thermography_split"


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


In [ ]:
import os
print(os.listdir("/content/drive"))
print(os.listdir("/content/drive/MyDrive"))


In [ ]:
import os

for root, dirs, files in os.walk("/content/drive"):
    if "BCD_Thermography_split" in dirs:
        print("FOUND:", os.path.join(root, "BCD_Thermography_split"))
        break


In [ ]:
base_path = "/content/drive/MyDrive/BCD_Thermography_split"


In [ ]:
import os
print(os.listdir(base_path))
print(os.listdir(base_path + "/train"))
print(os.listdir(base_path + "/val"))


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input


In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    shear_range=0.1,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)


In [ ]:
base_path = "/content/drive/MyDrive/BCD_Thermography_split"


In [ ]:
img_size = (224,224)
batch_size = 8

train_gen = train_datagen.flow_from_directory(
    base_path + "/train",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    base_path + "/val",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    shuffle=False
)

print(train_gen.class_indices)
print(val_gen.class_indices)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0

base_model = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3)
)

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

model.summary()


In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)


In [ ]:
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix, classification_report, roc_auc_score

val_gen.reset()
y_prob = model.predict(val_gen).ravel()
y_true = val_gen.classes

best_t = 0.5
best_f1 = 0

for t in np.linspace(0.05, 0.95, 19):
    y_pred = (y_prob > t).astype(int)
    f1 = f1_score(y_true, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_t = t

print("Best Threshold:", best_t)
y_pred = (y_prob > best_t).astype(int)

print("VAL AUC:", roc_auc_score(y_true, y_prob))
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=["NORMAL","SICK"]))


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3)
)
base_model.trainable = False

inputs = layers.Input(shape=(224,224,3))
x = base_model(inputs, training=False)

# (batch, 7, 7, 1280) -> reshape for attention
x = layers.Reshape((7*7, 1280))(x)

# Self Attention
attn = layers.MultiHeadAttention(num_heads=4, key_dim=128)(x, x)
x = layers.Add()([x, attn])
x = layers.LayerNormalization()(x)

x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

model.summary()


In [ ]:
early = tf.keras.callbacks.EarlyStopping(
    monitor="val_auc",
    patience=4,
    mode="max",
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_auc",
    factor=0.5,
    patience=2,
    mode="max",
    min_lr=1e-7
)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=25,
    callbacks=[early, reduce_lr]
)


In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
import numpy as np

val_gen.reset()
y_prob = model.predict(val_gen).ravel()
y_true = val_gen.classes

print("VAL AUC:", roc_auc_score(y_true, y_prob))

best_t = 0.5
best_f1 = 0
for t in np.linspace(0.05, 0.95, 19):
    y_pred = (y_prob > t).astype(int)
    f1 = (2*( (y_pred & (y_true==1)).sum() )) / (y_pred.sum() + (y_true==1).sum() + 1e-9)
    if f1 > best_f1:
        best_f1 = f1
        best_t = t

print("Best Threshold:", best_t)
y_pred = (y_prob > best_t).astype(int)

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=["NORMAL","SICK"]))


In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score

val_gen.reset()
y_prob = model.predict(val_gen).ravel()
y_true = val_gen.classes

best_t, best_f1 = 0.5, 0
for t in np.arange(0.05, 0.95, 0.05):
    y_pred = (y_prob > t).astype(int)
    f1 = f1_score(y_true, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_t = t

print("BEST THRESHOLD:", best_t)
y_pred = (y_prob > best_t).astype(int)

print("VAL AUC:", roc_auc_score(y_true, y_prob))
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=["NORMAL","SICK"]))


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(train_gen.classes)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=train_gen.classes)
class_weight = dict(zip(classes, weights))
print(class_weight)


In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=25,
    class_weight=class_weight,
    callbacks=[early, reduce_lr]
)


In [ ]:
base_model.trainable = False


In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)


In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.1,
    brightness_range=[0.9, 1.1]
)


In [ ]:
val_gen.reset()
y_prob = model.predict(val_gen, verbose=0).ravel()
y_true = val_gen.classes

from sklearn.metrics import roc_auc_score, average_precision_score
print("ROC AUC:", roc_auc_score(y_true, y_prob))
print("PR AUC:", average_precision_score(y_true, y_prob))


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

base_model.trainable = False   # pehle freeze

inputs = tf.keras.Input(shape=(224,224,3))
x = base_model(inputs, training=False)   # EfficientNet output (7,7,1280)

# --- Attention Hybrid Block ---
x = layers.Reshape((49, 1280))(x)

attn = layers.MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
x = layers.Add()([x, attn])
x = layers.LayerNormalization()(x)

# --- Pooling + Strong Regularized Head ---
x = layers.GlobalAveragePooling1D()(x)

x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation="relu",
                 kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
x = layers.Dropout(0.4)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model = Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

model.summary()


In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=25,
    class_weight=class_weight,
    callbacks=[early, reduce_lr]
)


In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score

val_gen.reset()
y_prob = model.predict(val_gen, verbose=0).ravel()
y_true = val_gen.classes

print("ROC AUC:", roc_auc_score(y_true, y_prob))
print("PR AUC:", average_precision_score(y_true, y_prob))


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=[
        "accuracy",
        tf.keras.metrics.AUC(name="roc_auc"),
        tf.keras.metrics.AUC(name="pr_auc", curve="PR")
    ]
)


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

ckpt = ModelCheckpoint(
    "best_model.keras",
    monitor="val_auc",
    mode="max",
    save_best_only=True,
    verbose=1
)


In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=25,
    callbacks=[early, reduce_lr, ckpt],
    class_weight=class_weight
)


In [ ]:
model = tf.keras.models.load_model("best_model.keras")


In [ ]:
base_model.trainable = False
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="binary_crossentropy",
              metrics=["accuracy", tf.keras.metrics.AUC(name="auc")])
model.fit(train_gen, validation_data=val_gen, epochs=5)


In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss="binary_crossentropy",
              metrics=["accuracy", tf.keras.metrics.AUC(name="auc")])

model.fit(train_gen, validation_data=val_gen, epochs=20,
          callbacks=[early, reduce_lr, ckpt],
          class_weight=class_weight)


In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=8,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.08,
    brightness_range=[0.95, 1.05],
    horizontal_flip=True
)


In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score, classification_report, confusion_matrix

val_gen.reset()
y_prob = model.predict(val_gen).ravel()
y_true = val_gen.classes

print("ROC AUC:", roc_auc_score(y_true, y_prob))
print("PR AUC:", average_precision_score(y_true, y_prob))


In [ ]:
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.applications.efficientnet import preprocess_input


In [ ]:
img_size = (260, 260)


In [ ]:
img_size = (300, 300)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.applications.efficientnet import preprocess_input

img_size = (260,260)

inp = layers.Input(shape=(img_size[0], img_size[1], 3))
base = EfficientNetB2(include_top=False, weights="imagenet", input_tensor=inp)
base.trainable = False

x = base.output                          # (None, H, W, C)
x = layers.Reshape((-1, x.shape[-1]))(x) # (None, tokens, C)

attn = layers.MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
x = layers.Add()([x, attn])
x = layers.LayerNormalization()(x)

x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.3)(x)
out = layers.Dense(1, activation="sigmoid")(x)

model = Model(inp, out)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

model.summary()


In [ ]:
val_dir = "/content/drive/MyDrive/BCD_Thermography_split/val"
train_dir = "/content/drive/MyDrive/BCD_Thermography_split/train"
test_dir  = "/content/drive/MyDrive/BCD_Thermography_split/test"


In [ ]:
val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(260,260),
    batch_size=8,
    class_mode="binary",
    shuffle=False
)


In [ ]:
import os
print(os.path.exists(val_dir), val_dir)
print(os.listdir(val_dir))


In [ ]:
val_gen.reset()
y_prob = model.predict(val_gen, verbose=0).ravel()
y_true = val_gen.classes


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(260,260),
    batch_size=8,
    class_mode="binary",
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(260,260),
    batch_size=8,
    class_mode="binary",
    shuffle=False
)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, GlobalAveragePooling1D, Dense, Dropout, Reshape, Add, LayerNormalization
from tensorflow.keras.applications import EfficientNetB2

inp = Input(shape=(260,260,3))
base = EfficientNetB2(include_top=False, weights="imagenet", input_tensor=inp)
base.trainable = False

x = base.output                         # (None, 9, 9, 1408)
x = Reshape((81, 1408))(x)

attn = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
x = Add()([x, attn])
x = LayerNormalization()(x)

x = GlobalAveragePooling1D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)

out = Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inp, out)


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=[
        tf.keras.metrics.AUC(name="roc_auc"),
        tf.keras.metrics.AUC(name="pr_auc", curve="PR"),
        "accuracy"
    ]
)


In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        "best_model.keras",
        monitor="val_roc_auc",
        mode="max",
        save_best_only=True,
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_roc_auc",
        mode="max",
        patience=5,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_roc_auc",
        mode="max",
        factor=0.5,
        patience=2,
        min_lr=1e-6,
        verbose=1
    )
]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=25,
    callbacks=callbacks
)


In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score

val_gen.reset()
y_prob = model.predict(val_gen, verbose=0).ravel()
y_true = val_gen.classes

print("ROC AUC:", roc_auc_score(y_true, y_prob))
print("PR AUC :", average_precision_score(y_true, y_prob))


In [ ]:
import numpy as np
from sklearn.metrics import f1_score

best_t = 0.5
best_f1 = 0

for t in np.arange(0.1, 0.9, 0.05):
    y_pred = (y_prob > t).astype(int)
    f1 = f1_score(y_true, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_t = t

print("Best Threshold:", best_t, "Best F1:", best_f1)


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = (y_prob > best_t).astype(int)
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=["NORMAL","SICK"]))


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.array([0,1]),
    y=train_gen.classes
)
class_weight = {0: class_weights[0], 1: class_weights[1]}
print(class_weight)


In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=25,
    callbacks=callbacks,
    class_weight=class_weight
)


In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    shear_range=0.1,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True
)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=8,
    class_mode="binary",
    shuffle=True
)


In [ ]:
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224,224),
    batch_size=8,
    class_mode="binary",
    shuffle=False
)


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.array([0,1]),
    y=train_gen.classes
)
class_weight = {0: class_weights[0], 1: class_weights[1]}
print(class_weight)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model

base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224,224,3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
out = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=out)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="roc_auc")]
)
model.summary()


In [ ]:
early = tf.keras.callbacks.EarlyStopping(monitor="val_roc_auc", patience=5, mode="max", restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_roc_auc", factor=0.5, patience=2, mode="max")

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=25,
    class_weight=class_weight,
    callbacks=[early, reduce_lr]
)


In [ ]:
val_gen.reset()
y_prob = model.predict(val_gen, verbose=0).ravel()
y_true = val_gen.classes


In [ ]:
import numpy as np
from sklearn.metrics import f1_score

best_t = 0.5
best_f1 = 0

for t in np.arange(0.05, 0.95, 0.05):
    y_pred = (y_prob > t).astype(int)
    f1 = f1_score(y_true, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_t = t

print("Best Threshold:", best_t, "Best F1:", best_f1)


In [ ]:
class_weight = {0: float(class_weights[0]), 1: float(class_weights[1])}


In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.1,
    brightness_range=[0.9, 1.1],
    horizontal_flip=True
)


In [ ]:
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=25,
    class_weight=class_weight,
    callbacks=[early, reduce_lr]
)


In [ ]:
import os
print(os.listdir("/content/drive/MyDrive/BCD_Thermography_split"))


In [ ]:
test_gen = val_gen


In [ ]:
train_dir = "/content/drive/MyDrive/BCD_Thermography_split/train"
val_dir   = "/content/drive/MyDrive/BCD_Thermography_split/val"


In [ ]:
import os, random, shutil

base = "/content/drive/MyDrive/BCD_Thermography_split/train"
normal_dir = os.path.join(base, "NORMAL")
sick_dir   = os.path.join(base, "SICK")

normal_imgs = os.listdir(normal_dir)
sick_imgs   = os.listdir(sick_dir)

print("Before Balance:", len(normal_imgs), len(sick_imgs))

target = min(len(normal_imgs), len(sick_imgs))

# Random select only target normal images
selected_normal = random.sample(normal_imgs, target)

# Create new balanced folder
balanced_base = "/content/balanced_train"
balanced_normal = os.path.join(balanced_base, "NORMAL")
balanced_sick   = os.path.join(balanced_base, "SICK")

os.makedirs(balanced_normal, exist_ok=True)
os.makedirs(balanced_sick, exist_ok=True)

# Copy selected normal
for img in selected_normal:
    shutil.copy(os.path.join(normal_dir, img), os.path.join(balanced_normal, img))

# Copy all sick
for img in sick_imgs:
    shutil.copy(os.path.join(sick_dir, img), os.path.join(balanced_sick, img))

print("After Balance:", len(os.listdir(balanced_normal)), len(os.listdir(balanced_sick)))


In [ ]:
train_gen = train_datagen.flow_from_directory(
    "/content/balanced_train",
    target_size=(224,224),
    batch_size=8,
    class_mode="binary",
    shuffle=True
)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

IMG_SIZE = (224,224)
BATCH = 8

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    shear_range=0.1,
    brightness_range=[0.8,1.2],
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen = train_datagen.flow_from_directory(
    "/content/balanced_train",   # balanced wala
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    "/content/drive/MyDrive/BCD_Thermography_split/val",
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    shuffle=False
)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = EfficientNetB3(weights="imagenet", include_top=False, input_shape=(224,224,3))
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.4)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
out = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=out)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="roc_auc")]
)


In [ ]:
early = tf.keras.callbacks.EarlyStopping(monitor="val_roc_auc", patience=5, mode="max", restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_roc_auc", patience=2, factor=0.5, mode="max")

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=25,
    callbacks=[early, reduce_lr]
)


In [ ]:
import os
import tensorflow as tf


In [ ]:
train_dir = "/content/balanced_train"
val_dir   = "/content/drive/MyDrive/BCD_Thermography_split/val"

print("train exists:", os.path.exists(train_dir), train_dir)
print("val exists:", os.path.exists(val_dir), val_dir)

print("train folders:", os.listdir(train_dir) if os.path.exists(train_dir) else "NOT FOUND")
print("val folders:", os.listdir(val_dir) if os.path.exists(val_dir) else "NOT FOUND")


In [ ]:
IMG_SIZE = (224,224)
BATCH = 8

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    shear_range=0.1,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    shuffle=False
)

print("Class mapping:", train_gen.class_indices)


In [ ]:
import tensorflow as tf

early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    min_lr=1e-7,
    verbose=1
)


In [ ]:
import tensorflow as tf


In [ ]:
early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    min_lr=1e-7,
    verbose=1
)


In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    min_lr=1e-7,
    verbose=1
)


In [ ]:
print("model:", "OK" if "model" in globals() else "MISSING")
print("train_gen:", "OK" if "train_gen" in globals() else "MISSING")
print("val_gen:", "OK" if "val_gen" in globals() else "MISSING")
print("early:", "OK" if "early" in globals() else "MISSING")
print("reduce_lr:", "OK" if "reduce_lr" in globals() else "MISSING")


In [ ]:
import tensorflow as tf
import numpy as np
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB3, preprocess_input
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model


In [ ]:
train_dir = "/content/balanced_train"   # tumhara balanced folder
val_dir   = "/content/drive/MyDrive/BCD_Thermography_split/val"

IMG_SIZE = (224,224)
BATCH = 8


In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    shear_range=0.1,
    brightness_range=[0.8,1.2],
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    shuffle=False
)

print("Class mapping:", train_gen.class_indices)


In [ ]:
import os
print("train_dir:", train_dir)
print("Exists?", os.path.exists(train_dir))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
train_dir = "/content/drive/MyDrive/Breast_Cancer_Dataset/Train"
test_dir  = "/content/drive/MyDrive/Breast_Cancer_Dataset/Test"

IMG_SIZE = (224,224)
BATCH = 8


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    shuffle=False
)

print(train_gen.class_indices)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = EfficientNetB3(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.4)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
out = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=out)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="roc_auc")]
)


In [ ]:
early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    min_lr=1e-7,
    verbose=1
)


In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=25,
    callbacks=[early, reduce_lr],
    verbose=1
)


In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="roc_auc")]
)


In [ ]:
history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[early, reduce_lr]
)


In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False


In [ ]:
# Best Threshold ≈ 0.55–0.6


In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False



In [ ]:
loss = tf.keras.losses.BinaryFocalCrossentropy(gamma=2.0)


In [ ]:
model.fit(
    train_gen,
    epochs=25,
    callbacks=[reduce_lr],
    verbose=1
)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model


In [ ]:
print(tf.__version__)


In [ ]:
TRAIN_DIR = "/content/drive/MyDrive/Breast_Cancer_Dataset/Train"
TEST_DIR  = "/content/drive/MyDrive/Breast_Cancer_Dataset/Test"

IMG_SIZE = (224, 224)
BATCH = 16


In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    shuffle=True
)


In [ ]:
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_gen = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    shuffle=False
)

print("Class mapping:", train_gen.class_indices)


In [ ]:
base_model = EfficientNetB3(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.4)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(base_model.input, output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

model.summary()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model


In [ ]:
print(tf.__version__)


In [ ]:
TRAIN_DIR = "/content/drive/MyDrive/Breast_Cancer_Dataset/Train"
TEST_DIR  = "/content/drive/MyDrive/Breast_Cancer_Dataset/Test"

IMG_SIZE = (224, 224)
BATCH = 16


In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    shuffle=True
)


In [ ]:
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_gen = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    shuffle=False
)

print("Class mapping:", train_gen.class_indices)


In [ ]:
base_model = EfficientNetB3(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.4)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(base_model.input, output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

model.summary()


In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="loss",
    factor=0.3,
    patience=2,
    min_lr=1e-7,
    verbose=1
)


In [ ]:
history = model.fit(
    train_gen,
    epochs=25,
    callbacks=[reduce_lr],
    verbose=1
)


In [ ]:
base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

history_fine = model.fit(
    train_gen,
    epochs=15,
    callbacks=[reduce_lr],
    verbose=1
)


In [ ]:
test_gen.reset()
y_prob = model.predict(test_gen).ravel()
y_true = test_gen.classes


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

best_acc = 0
best_t = 0.5

for t in np.arange(0.3, 0.8, 0.05):
    y_pred = (y_prob > t).astype(int)
    acc = accuracy_score(y_true, y_pred)
    if acc > best_acc:
        best_acc = acc
        best_t = t

print("Best Threshold:", best_t)
print("Best Accuracy:", best_acc)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = (y_prob > best_t).astype(int)

print("Accuracy:", accuracy_score(y_true, y_pred))
print("F1 Score:", f1_score(y_true, y_pred))
print("AUC:", roc_auc_score(y_true, y_prob))

print(classification_report(y_true, y_pred))


In [ ]:
plt.figure(figsize=(6,5))
plt.plot(history.history["accuracy"], label="Train Acc")
plt.title("Training Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import f1_score

best_f1 = 0
best_t = 0.5

for t in np.arange(0.2, 0.7, 0.02):
    y_pred = (y_prob > t).astype(int)
    f1 = f1_score(y_true, y_pred)

    if f1 > best_f1:
        best_f1 = f1
        best_t = t

print("Best Threshold:", best_t)
print("Best F1:", best_f1)


In [ ]:
from tensorflow.keras.layers import Multiply, Dense

def channel_attention(x, ratio=8):
    channels = x.shape[-1]

    shared = Dense(channels//ratio, activation='relu')
    excite = Dense(channels, activation='sigmoid')

    attn = excite(shared(x))
    return Multiply()([x, attn])


In [ ]:
x = GlobalAveragePooling2D()(base_model.output)
x = channel_attention(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
out = Dense(1, activation='sigmoid')(x)


In [ ]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)

class_weight = dict(enumerate(class_weights))


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.utils.class_weight import compute_class_weight


In [ ]:
IMG_SIZE = (224,224)
BATCH = 8

train_dir = "/content/drive/MyDrive/Breast_Cancer_Dataset/Train"
val_dir   = "/content/drive/MyDrive/Breast_Cancer_Dataset/Test"

train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=15,
    zoom_range=0.15,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    shuffle=False
)


In [ ]:
IMG_SIZE = (224,224)
BATCH = 8

train_dir = "/content/drive/MyDrive/Breast_Cancer_Dataset/Train"
val_dir   = "/content/drive/MyDrive/Breast_Cancer_Dataset/Test"

train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=15,
    zoom_range=0.15,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    shuffle=False
)


In [ ]:
base_model = EfficientNetB3(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation="relu")(x)
x = Dropout(0.4)(x)
out = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=out)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)


In [ ]:
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)

class_weight = dict(enumerate(class_weights))
print(class_weight)


In [ ]:
early = tf.keras.callbacks.EarlyStopping(
    monitor="val_auc",
    patience=6,
    mode="max",
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_auc",
    factor=0.3,
    patience=3,
    mode="max",
    verbose=1
)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    class_weight=class_weight,
    callbacks=[early, reduce_lr]
)


In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import (
    GlobalAveragePooling2D, GlobalMaxPooling2D,
    Dense, Reshape, Multiply, Add, Conv2D,
    Activation, Lambda
)

def cbam_block(feature_map, ratio=8):
    channel = K.int_shape(feature_map)[-1]

    # ---------- Channel Attention ----------
    avg_pool = GlobalAveragePooling2D()(feature_map)
    max_pool = GlobalMaxPooling2D()(feature_map)

    shared_dense1 = Dense(channel // ratio, activation='relu')
    shared_dense2 = Dense(channel)

    avg_out = shared_dense2(shared_dense1(avg_pool))
    max_out = shared_dense2(shared_dense1(max_pool))

    channel_attention = Activation('sigmoid')(Add()([avg_out, max_out]))
    channel_attention = Reshape((1,1,channel))(channel_attention)

    channel_refined = Multiply()([feature_map, channel_attention])

    # ---------- Spatial Attention (FIXED) ----------
    avg_pool_spatial = Lambda(
        lambda x: tf.reduce_mean(x, axis=-1, keepdims=True)
    )(channel_refined)

    max_pool_spatial = Lambda(
        lambda x: tf.reduce_max(x, axis=-1, keepdims=True)
    )(channel_refined)

    spatial_concat = Lambda(
        lambda x: tf.concat(x, axis=-1)
    )([avg_pool_spatial, max_pool_spatial])

    spatial_attention = Conv2D(
        1, kernel_size=7, padding="same", activation="sigmoid"
    )(spatial_concat)

    refined_feature = Multiply()([channel_refined, spatial_attention])

    return refined_feature


In [ ]:
base_model.trainable = False

x = base_model.output
x = cbam_block(x)                 # ✅ NOW NO ERROR
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=output)


In [ ]:
model.summary()


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=[
        "accuracy",
        tf.keras.metrics.AUC(name="auc")
    ]
)


In [ ]:
history_cbam = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    class_weight=class_weight,
    callbacks=[early, reduce_lr]
)


In [ ]:
base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)


In [ ]:
best_acc = 0
best_t = 0.5

for t in np.arange(0.3, 0.8, 0.05):
    y_pred = (y_prob > t).astype(int)
    acc = accuracy_score(y_true, y_pred)
    if acc > best_acc:
        best_acc = acc
        best_t = t

print("Best Threshold:", best_t)
print("Best Accuracy:", best_acc)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = (y_prob > best_t).astype(int)

print(classification_report(y_true, y_pred))
print("AUC:", roc_auc_score(y_true, y_prob))


In [ ]:
import matplotlib.pyplot as plt

# Accuracy
plt.figure()
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Loss
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix

y_pred = (y_prob > best_t).astype(int)

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d',
            xticklabels=['Benign','Malignant'],
            yticklabels=['Benign','Malignant'],
            cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix (Test Set)')
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(y_true, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, label=f'Proposed Model (AUC = {roc_auc:.4f})')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve

precision, recall, _ = precision_recall_curve(y_true, y_prob)

plt.figure()
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision–Recall Curve')
plt.grid(True)
plt.show()


In [ ]:
import tensorflow as tf
import cv2

def gradcam(model, img_array, layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, 0]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.reduce_max(heatmap)
    return heatmap.numpy()


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.figure(figsize=(14,5))

# Accuracy
plt.subplot(1,2,1)
plt.plot(epochs, acc, label='Train Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy')
plt.legend()
plt.grid(True)

# Loss
plt.subplot(1,2,2)
plt.plot(epochs, loss, label='Train Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()
plt.grid(True)

plt.show()


In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6,5))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=['Benign','Malignant'],
    yticklabels=['Benign','Malignant']
)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix (Test Set)')
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(y_true, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, label=f'Proposed Attention-CNN (AUC = {roc_auc:.4f})')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve

precision, recall, _ = precision_recall_curve(y_true, y_prob)

plt.figure(figsize=(6,5))
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision–Recall Curve')
plt.grid(True)
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

def get_attention_map(model, img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224,224))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    x = img_rgb / 255.0
    x = np.expand_dims(x, axis=0)

    conv_layer = model.get_layer(index=-6).output  # CBAM se pehle wala layer
    heatmap_model = tf.keras.models.Model(model.input, conv_layer)

    feature_maps = heatmap_model.predict(x)
    heatmap = np.mean(feature_maps[0], axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    heatmap = cv2.resize(heatmap, (224,224))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed = cv2.addWeighted(img_rgb, 0.6, heatmap, 0.4, 0)

    return img_rgb, superimposed


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

def get_attention_map(model, img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224,224))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    x = img_rgb / 255.0
    x = np.expand_dims(x, axis=0)

    conv_layer = model.get_layer(index=-6).output  # CBAM se pehle wala layer
    heatmap_model = tf.keras.models.Model(model.input, conv_layer)

    feature_maps = heatmap_model.predict(x)
    heatmap = np.mean(feature_maps[0], axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    heatmap = cv2.resize(heatmap, (224,224))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed = cv2.addWeighted(img_rgb, 0.6, heatmap, 0.4, 0)

    return img_rgb, superimposed


In [ ]:
BASE_PATH = "/content/drive/MyDrive/Breast_Cancer_Dataset"
print(os.listdir(BASE_PATH))


In [ ]:
test_path = "/content/drive/MyDrive/Breast_Cancer_Dataset/Test/Malignant"
print(os.listdir(test_path)[:10])


In [ ]:
IMG_PATH = "/content/drive/MyDrive/Breast_Cancer_Dataset/Test/Malignant/PAC_11_DN4-dir.png"


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

def get_attention_heatmap(model, img_path):
    # Load image
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    x = img_rgb / 255.0
    x = np.expand_dims(x, axis=0)

    # 🔥 LAST CONVOLUTIONAL LAYER AUTO-DETECT
    last_conv_layer = None
    for layer in reversed(model.layers):
        if len(layer.output_shape) == 4:
            last_conv_layer = layer
            break

    grad_model = tf.keras.models.Model(
        [model.inputs],
        [last_conv_layer.output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(x)
        loss = predictions[:, 0]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(conv_outputs * pooled_grads, axis=-1)

    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    heatmap = cv2.resize(heatmap, (224, 224))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed = cv2.addWeighted(img_rgb, 0.6, heatmap, 0.4, 0)

    return img_rgb, superimposed


In [ ]:
model.summary()


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

def get_attention_heatmap(model, img_path, last_conv_layer_name):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224,224))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    x = img_rgb / 255.0
    x = np.expand_dims(x, axis=0)

    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(x)
        loss = predictions[:, 0]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    heatmap = cv2.resize(heatmap, (224,224))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed = cv2.addWeighted(img_rgb, 0.6, heatmap, 0.4, 0)

    return img_rgb, superimposed


In [ ]:
# Sab conv layers ke naam dekho
for layer in model.layers:
    if 'conv' in layer.name.lower():
        print(layer.name)


In [ ]:
LAST_CONV = "conv2d_4"   # ⚠️ agar tumhare output me different ho, to wahi likhna


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

def get_attention_heatmap(model, img_path, last_conv_layer):
    # Load image
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    x = img_rgb / 255.0
    x = np.expand_dims(x, axis=0)

    # Gradient model
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(x)
        loss = predictions[:, 0]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    heatmap = cv2.resize(heatmap.numpy(), (224, 224))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed = cv2.addWeighted(img_rgb, 0.6, heatmap, 0.4, 0)

    return img_rgb, superimposed


In [ ]:
IMG_PATH = "/content/drive/MyDrive/Breast_Cancer_Dataset/Test/Malignant/PAC_11_DN4-dir.png"


In [ ]:
import os

test_dir = "/content/drive/MyDrive/Breast_Cancer_Dataset/Test/Malignant"
print(os.listdir(test_dir)[:10])


In [ ]:
def get_attention_heatmap(model, img_path, last_conv_layer):
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Image not found: {img_path}")

    img = cv2.resize(img, (224, 224))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    x = img_rgb / 255.0
    x = np.expand_dims(x, axis=0)

    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_out, preds = grad_model(x)
        loss = preds[:, 0]

    grads = tape.gradient(loss, conv_out)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_out = conv_out[0]
    heatmap = conv_out @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    heatmap = cv2.resize(heatmap.numpy(), (224, 224))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed = cv2.addWeighted(img_rgb, 0.6, heatmap, 0.4, 0)

    return img_rgb, superimposed


In [ ]:
IMG_PATH = "/content/drive/MyDrive/Breast_Cancer_Dataset/Test/Malignant/PAC_14_DN0-dir.png"


In [ ]:
def get_attention_heatmap(model, img_path, last_conv_layer):
    import cv2, numpy as np, tensorflow as tf

    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Image not found: {img_path}")

    img = cv2.resize(img, (224, 224))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    x = img_rgb / 255.0
    x = np.expand_dims(x, axis=0)

    grad_model = tf.keras.models.Model(
        model.inputs,
        [model.get_layer(last_conv_layer).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_out, preds = grad_model(x)
        loss = preds[:, 0]

    grads = tape.gradient(loss, conv_out)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_out = conv_out[0]
    heatmap = tf.reduce_sum(conv_out * pooled_grads, axis=-1)

    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    heatmap = cv2.resize(heatmap, (224, 224))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed = cv2.addWeighted(img_rgb, 0.6, heatmap, 0.4, 0)

    return img_rgb, superimposed


In [ ]:
import tensorflow as tf

def find_last_conv_layer(model):
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            return layer.name
    raise ValueError("No Conv2D layer found")

LAST_CONV = find_last_conv_layer(model)
print("Using last conv layer:", LAST_CONV)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

def get_attention_heatmap(model, img_path, last_conv_layer):
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Image not found: {img_path}")

    img = cv2.resize(img, (224, 224))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    x = img_rgb / 255.0
    x = np.expand_dims(x, axis=0)

    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_out, preds = grad_model(x)
        loss = preds[:, 0]

    grads = tape.gradient(loss, conv_out)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_out = conv_out[0]
    heatmap = tf.reduce_sum(conv_out * pooled_grads, axis=-1)

    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) + 1e-8
    heatmap = cv2.resize(heatmap, (224, 224))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed = cv2.addWeighted(img_rgb, 0.6, heatmap, 0.4, 0)
    return img_rgb, superimposed


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
test_dir = "/content/drive/MyDrive/Breast_Cancer_Dataset/Test/Malignant"
print(os.listdir(test_dir)[:5])


['PAC_14_DN0-dir.png', 'PAC_14_DN10-dir.png', 'PAC_14_DN1-esq.png', 'PAC_14_DN10-esq.png', 'PAC_14_DN1-dir.png']


In [ ]:
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 224, 224,  │          0 │ input_layer_2[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 224, 224,  │          7 │ rescaling_2[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (None, 224, 224,  │          0 │ normalization_1[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_3[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,050,852 (15.45 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
import tensorflow as tf

def find_last_conv_layer(model):
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            return layer.name
    raise ValueError("No Conv2D layer found")

LAST_CONV = find_last_conv_layer(model)
print("Using last conv layer:", LAST_CONV)


Using last conv layer: top_conv


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

def get_attention_heatmap(model, img_path, last_conv_layer):
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Image not found: {img_path}")

    img = cv2.resize(img, (224, 224))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    x = img_rgb / 255.0
    x = np.expand_dims(x, axis=0)

    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_out, preds = grad_model(x)
        loss = preds[:, 0]

    grads = tape.gradient(loss, conv_out)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_out = conv_out[0]
    heatmap = conv_out @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) + 1e-8

    heatmap = cv2.resize(heatmap, (224, 224))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed = cv2.addWeighted(img_rgb, 0.6, heatmap, 0.4, 0)

    return img_rgb, superimposed


In [ ]:
IMG_PATH = "/content/drive/MyDrive/Breast_Cancer_Dataset/Test/Malignant/PAC_14_DN0-dir.png"

original, attention = get_attention_heatmap(
    model,
    IMG_PATH,
    LAST_CONV
)

plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.imshow(original)
plt.title("Original Thermogram")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(attention)
plt.title("Attention Heatmap (Proposed Attention-Based Hybrid CNN)")
plt.axis("off")

plt.tight_layout()
plt.show()


NameError: name 'get_attention_heatmap' is not defined

In [ ]:
import os
os.makedirs("results", exist_ok=True)
print(os.listdir("results"))


['roc_curve.png']


In [ ]:
plt.savefig("results/attention_heatmap.png", dpi=300, bbox_inches="tight")


<Figure size 640x480 with 0 Axes>

In [ ]:



































import os
os.makedirs("results", exist_ok=True)
print(os.listdir())


['.config', 'drive', 'results', 'sample_data']


In [1]:
import os
os.makedirs("results", exist_ok=True)


In [2]:
from IPython.display import Image, display

display(Image("results/roc_curve.png"))
display(Image("results/confusion_matrix.png"))
display(Image("results/precision_recall.png"))
display(Image("results/learning_curves.png"))
display(Image("results/attention_heatmap.png"))


FileNotFoundError: No such file or directory: 'results/roc_curve.png'

FileNotFoundError: No such file or directory: 'results/roc_curve.png'

<IPython.core.display.Image object>

FileNotFoundError: No such file or directory: 'results/confusion_matrix.png'

FileNotFoundError: No such file or directory: 'results/confusion_matrix.png'

...

FileNotFoundError: No such file or directory: 'results/learning_curves.png'

<IPython.core.display.Image object>

FileNotFoundError: No such file or directory: 'results/attention_heatmap.png'

FileNotFoundError: No such file or directory: 'results/attention_heatmap.png'

<IPython.core.display.Image object>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
